In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

**Definicion de variables**

In [2]:
cant_particiones = 5
ventana = "auctions_2019-04-18-2019-04-20"
ventana2 = "auctions_2019-04-21-2019-04-23"
carpeta_data = "data/"
ruta = carpeta_data + ventana
ruta2 = carpeta_data + ventana2
extension = ".csv"
dia_inicio = 18
dia_inicio_v2 = 21
tiempo_max = 60*60*24*3

In [3]:
reg = MLPRegressor()
ult_por_user = pd.DataFrame(columns=["date", "device_id", "ref_type_id", "source_id"])
test_y = pd.read_csv("data/test_t.csv")

**Se crea el Dataframe de test**

In [4]:
"""for i in range(cant_particiones):
    test_y_temp = pd.read_csv(f"{ruta2}-users{i}{extension}", low_memory=False, parse_dates = ['date'], index_col=['date'])
    test_y_temp = test_y_temp.reset_index()
    test_y_temp['date'] = test_y_temp['date'] - pd.Timestamp(f'2019-04-{dia_inicio_v2} 00:00:00')
    test_y_temp['date'] = test_y_temp['date'].transform(lambda x: x.total_seconds())
    test_y_temp = test_y_temp.groupby('device_id').agg({'date' : 'min'}).reset_index()
    test_y_temp = test_y_temp[["device_id", "date"]]
    test_y = pd.concat([test_y, test_y_temp])
"""

'for i in range(cant_particiones):\n    test_y_temp = pd.read_csv(f"{ruta2}-users{i}{extension}", low_memory=False, parse_dates = [\'date\'], index_col=[\'date\'])\n    test_y_temp = test_y_temp.reset_index()\n    test_y_temp[\'date\'] = test_y_temp[\'date\'] - pd.Timestamp(f\'2019-04-{dia_inicio_v2} 00:00:00\')\n    test_y_temp[\'date\'] = test_y_temp[\'date\'].transform(lambda x: x.total_seconds())\n    test_y_temp = test_y_temp.groupby(\'device_id\').agg({\'date\' : \'min\'}).reset_index()\n    test_y_temp = test_y_temp[["device_id", "date"]]\n    test_y = pd.concat([test_y, test_y_temp])\n'

In [5]:
# test_y.to_csv("test_t.csv", index=False)

**Se entrena el algoritmo incrementalmente con cada uno de los 5 archivos**

In [6]:
for i in range(cant_particiones):
    train_x = pd.read_csv(f"{ruta}-users{i}{extension}", low_memory=False, parse_dates = ['date'], index_col=['date'])
    train_x = train_x.sort_index().reset_index()
    train_x['date'] = train_x['date'] - pd.Timestamp(f'2019-04-{dia_inicio} 00:00:00')
    train_x['date'] = train_x['date'].transform(lambda x: x.total_seconds())
    train_x = train_x.fillna(train_x.mode().iloc[0])
    train_x = train_x.sort_values(["device_id", "date"]).reset_index(drop=True)
    train_x["prox_ap"] = train_x.groupby("device_id")["date"].shift(-1)
    train_x["prox_ap"] = train_x["prox_ap"].fillna(tiempo_max)
    train_x["censored"] = (train_x["prox_ap"] == tiempo_max).transform(int)
    
    train_y = pd.Series(name="prox_ap")
    train_y = train_x["prox_ap"]
    train_x.drop(["prox_ap"], axis=1, inplace=True)

    ultimos = train_x.groupby("device_id").last().reset_index()
    ultimos = ultimos.merge(test_y["device_id"],how="inner")
    ult_por_user = pd.concat([ult_por_user, ultimos], ignore_index=True, sort=False)
    
    train_x_scaled = StandardScaler().fit_transform(train_x)
    reg.partial_fit(train_x_scaled, train_y)

/home/julian/.local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/julian/.local/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/julian/.local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/julian/.local/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/julian/.local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWar

In [7]:
ult_por_user = ult_por_user[["date", "device_id", "ref_type_id", "source_id", "censored"]]

In [8]:
ult_por_user = ult_por_user.merge(test_y["device_id"], how="left")

In [9]:
ult_por_user = ult_por_user.sort_values("device_id").reset_index(drop=True)

In [10]:
test_y = test_y.merge(ult_por_user["device_id"], how="inner")

In [11]:
test_y = test_y.sort_values("device_id").reset_index(drop=True)["date"]

In [12]:
ult_scaled = StandardScaler().fit_transform(ult_por_user)

/home/julian/.local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/julian/.local/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [13]:
preds = reg.predict(ult_scaled)

In [14]:
mean_squared_error(test_y, preds)**0.5

123774.18906907873

In [16]:
def cap(x):
    if x < 0:
        return 0
    elif x > 259200:
        return 259200
    else:
        return x

In [17]:
preds_c = list(map(cap, preds))

In [18]:
mean_squared_error(test_y, preds_c)**0.5

123774.18906907873

In [19]:
ult_comienzo = ult_por_user.copy()

In [20]:
ult_comienzo["date"] = 60*60*24*3

In [21]:
com_scaled = StandardScaler().fit_transform(ult_comienzo)

/home/julian/.local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/julian/.local/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [22]:
mi_pred = reg.predict(com_scaled)

In [23]:
mean_squared_error(test_y, mi_pred)**0.5

87677.03091746492

**El error es para 230000 usuarios**

**Hay que probar variar los parametros del regressor y ver que onda**